# HW #2, Introduction to Big Data Analytics, Fall 2019

## 2. Dimensionality reduction

### Exercise 11.1.7

In [1]:
import numpy as np
from numpy.linalg import norm, matrix_rank

def principal_eigenpair(M, epsilon=1e-6):
    '''
    Find principal eigenvector and eigenvalue using power iteration method.
    '''
    M = np.array(M)
    nCol, nRow = M.shape
    
    x = np.ones((nRow, 1))
    x_prev = np.full((nRow, 1), fill_value=np.inf)
    
    while norm(x - x_prev) > epsilon:
        p = np.matmul(M, x)
        x_prev, x = x, p/norm(p)
    
    l = np.matmul(np.matmul(x.T, M), x).flatten()
    
    return x, l

def subtract_eigenpair(M, e, l):
    return M - l*np.matmul(e, e.T)

def find_all_eigenpairs(M):
    M_ = []
    e_ = []
    l_ = []
    for _ in range(matrix_rank(M)):
        e, l = principal_eigenpair(M)
        M_.append(M)
        e_.append(e)
        l_.append(l)
        M = subtract_eigenpair(M, e, l)
    return M_, e_, l_

In [28]:
''' problem (a) and (b) '''
M = np.array([[1,1,1], [1,2,3], [1,3,6]])
e, l = principal_eigenpair(M)
print e, l

[[0.19382269]
 [0.4722473 ]
 [0.85989258]] [7.87298335]


In [29]:
''' problem (c) '''
print subtract_eigenpair(M, e, l)

[[ 0.7042338   0.2793682  -0.31216407]
 [ 0.2793682   0.24418683 -0.19707644]
 [-0.31216407 -0.19707644  0.17859602]]


In [30]:
''' problem (d) and (e) '''
print find_all_eigenpairs(M)

([array([[1, 1, 1],
       [1, 2, 3],
       [1, 3, 6]]), array([[ 0.7042338 ,  0.2793682 , -0.31216407],
       [ 0.2793682 ,  0.24418683, -0.19707644],
       [-0.31216407, -0.19707644,  0.17859602]]), array([[ 0.03756718, -0.05396498,  0.02116944],
       [-0.05396498,  0.0775203 , -0.03040975],
       [ 0.02116944, -0.03040975,  0.01192917]])], [array([[0.19382269],
       [0.4722473 ],
       [0.85989258]]), array([[ 0.81649655],
       [ 0.40824812],
       [-0.40824852]]), array([[ 0.54384354],
       [-0.78122728],
       [ 0.30646067]])], [array([7.87298335]), array([1.]), array([0.12701665])])


### Exercise 11.3.1

In [254]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
M = np.array([[1, 2, 3], 
              [3, 4, 5], 
              [5, 4, 3], 
              [0, 2, 4], 
              [1, 3, 5]])

In [255]:
''' problem (a) '''
mtm = np.matmul(M.T, M)
mmt = np.matmul(M, M.T)
print mtm
print mmt

[[36 37 38]
 [37 49 61]
 [38 61 84]]
[[14 26 22 16 22]
 [26 50 46 28 40]
 [22 46 50 20 32]
 [16 28 20 20 26]
 [22 40 32 26 35]]


In [262]:
''' problem (b) '''
# eigenpairs of MtM
mtm_eigvals, mtm_eigvecs = np.linalg.eig(mtm)
print mtm_eigvals
print mtm_eigvecs

# eigenpairs of MMt
mmt_eigvals, mmt_eigvecs = np.linalg.eig(mmt)
mmt_eigvals = np.real(mmt_eigvals)
mmt_eigvecs = np.real(mmt_eigvecs)
print mmt_eigvals
print mmt_eigvecs

[153.567  15.433   0.   ]
[[-0.409 -0.816  0.408]
 [-0.563 -0.126 -0.816]
 [-0.718  0.564  0.408]]
[153.567  -0.     15.433  -0.     -0.   ]
[[ 0.298  0.941  0.159  0.711  0.711]
 [ 0.571 -0.175 -0.033 -0.176 -0.176]
 [ 0.521 -0.04  -0.736  0.064  0.064]
 [ 0.323 -0.188  0.51   0.266  0.266]
 [ 0.459 -0.215  0.414 -0.502 -0.502]]


In [264]:
''' problem (c) '''
# construct V
V = mtm_eigvecs[:, [0,1]]

# construct U
U = mmt_eigvecs[:, [0,2]]
U[:, 0] = -U[:, 0]

# construct sigma
d = np.sqrt(mtm_eigvals[[0,1]])
sigma = np.diagflat(d)

print U
print sigma
print V

[[-0.298  0.159]
 [-0.571 -0.033]
 [-0.521 -0.736]
 [-0.323  0.51 ]
 [-0.459  0.414]]
[[12.392  0.   ]
 [ 0.     3.928]]
[[-0.409 -0.816]
 [-0.563 -0.126]
 [-0.718  0.564]]


In [267]:
'''problem (d)'''
V_ = V[:, 0:1]
U_ = U[:, 0:1]
sigma_ = sigma[0:1, 0:1]
M_ = np.matmul(np.matmul(U_, sigma_), V_.T)
print M_

[[1.51  2.079 2.647]
 [2.894 3.984 5.074]
 [2.641 3.636 4.631]
 [1.636 2.252 2.869]
 [2.328 3.205 4.082]]


In [268]:
'''problem (e)'''
retained_energy = (sigma[0,0]**2) / (sigma[0,0]**2 + sigma[1,1]**2)
print retained_energy

0.9086804524257934


### Exercise 11.4.2 (CUR decomposition)

In [341]:
import numpy as np
from numpy.linalg import eig, norm
M = np.array([[1,1,1,0,0], 
              [3,3,3,0,0], 
              [4,4,4,0,0], 
              [5,5,5,0,0], 
              [0,0,0,4,4], 
              [0,0,0,5,5], 
              [0,0,0,2,2]])

In [381]:
def CUR_verify(M, i, j, r=2):
    p = (np.sum(M**2, axis=1)/norm(M)**2).flatten() # distribution of i
    q = (np.sum(M**2, axis=0)/norm(M)**2).flatten() # distribution of j
    
    C = M[:, j]
    R = M[i, :]

    C = C / np.sqrt(r*q[j][np.newaxis, ...])
    R = R / np.sqrt(r*p[i][..., np.newaxis])

    W = M[i, :][:, j]

    X, s, Yh = np.linalg.svd(W)
    S = np.diagflat(s)
    Sp = np.linalg.pinv(S)
    print Sp
    U = np.matmul(np.matmul(Yh.T, np.matmul(Sp, Sp)), X.T)
    
    return C, U, R

In [408]:
'''problem (a)'''
i, j = [1,2], [0,1]

# construct C and R
p = (np.sum(M**2, axis=1)/norm(M)**2).flatten() # distribution of i
q = (np.sum(M**2, axis=0)/norm(M)**2).flatten() # distribution of j

C = M[:, j] / np.sqrt(r*q[j][np.newaxis, ...])
R = M[i, :] / np.sqrt(r*p[i][..., np.newaxis])

# construct U
W = M[i, :][:, j]

# SVD of W
d, Y = np.linalg.eig(np.matmul(W.T, W))
_, X = np.linalg.eig(np.matmul(W, W.T))
X = -X[:, [1,0]]
sigma = np.diagflat(np.sqrt(d))

# compute Moore-Penrose pseudoinverse of sigma
d[0] = 1/d[0]
sigma_p = np.diagflat(np.sqrt(d))

U = np.matmul(np.matmul(Y,np.matmul(sigma_p, sigma_p)), X.T)

print C
print U
print R

[[1.543 1.543]
 [4.63  4.63 ]
 [6.174 6.174]
 [7.717 7.717]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]]
[[0.008 0.011]
 [0.008 0.011]]
[[6.364 6.364 6.364 0.    0.   ]
 [6.364 6.364 6.364 0.    0.   ]]


In [409]:
'''problem (b)'''
i, j = [3,4], [1,2]

# construct C and R
p = (np.sum(M**2, axis=1)/norm(M)**2).flatten() # distribution of i
q = (np.sum(M**2, axis=0)/norm(M)**2).flatten() # distribution of j

C = M[:, j] / np.sqrt(r*q[j][np.newaxis, ...])
R = M[i, :] / np.sqrt(r*p[i][..., np.newaxis])

# construct U
W = M[i, :][:, j]

# SVD of W
d, Y = np.linalg.eig(np.matmul(W.T, W))
_, X = np.linalg.eig(np.matmul(W, W.T))
sigma = np.diagflat(np.sqrt(d))

# compute Moore-Penrose pseudoinverse of sigma
d[0] = 1/d[0]
sigma_p = np.diagflat(np.sqrt(d))

U = np.matmul(np.matmul(Y,np.matmul(sigma_p, sigma_p)), X.T)

print C
print U
print R

[[1.543 1.543]
 [4.63  4.63 ]
 [6.174 6.174]
 [7.717 7.717]
 [0.    0.   ]
 [0.    0.   ]
 [0.    0.   ]]
[[0.014 0.   ]
 [0.014 0.   ]]
[[6.364 6.364 6.364 0.    0.   ]
 [0.    0.    0.    7.794 7.794]]


In [410]:
'''problem (c)'''
i, j = [0,6], [0,4]

# construct C and R
p = (np.sum(M**2, axis=1)/norm(M)**2).flatten() # distribution of i
q = (np.sum(M**2, axis=0)/norm(M)**2).flatten() # distribution of j

C = M[:, j] / np.sqrt(r*q[j][np.newaxis, ...])
R = M[i, :] / np.sqrt(r*p[i][..., np.newaxis])

# construct U
W = M[i, :][:, j]

# SVD of W
d, Y = np.linalg.eig(np.matmul(W.T, W))
_, X = np.linalg.eig(np.matmul(W, W.T))
sigma = np.diagflat(np.sqrt(d))

# compute Moore-Penrose pseudoinverse of sigma
d = 1/d
sigma_p = np.diagflat(np.sqrt(d))

U = np.matmul(np.matmul(Y,np.matmul(sigma_p, sigma_p)), X.T)

print C
print U
print R

[[1.543 0.   ]
 [4.63  0.   ]
 [6.174 0.   ]
 [7.717 0.   ]
 [0.    6.573]
 [0.    8.216]
 [0.    3.286]]
[[1.   0.  ]
 [0.   0.25]]
[[6.364 6.364 6.364 0.    0.   ]
 [0.    0.    0.    7.794 7.794]]
